In [ ]:
from google.colab import files
def upload_file():
    """
    Загружает файл через интерфейс Colab.
    :return: Путь к загруженному файлу
    """
    print("Загрузите json с результатами...")
    uploaded = files.upload()

    if not uploaded:
        raise FileNotFoundError("Файл не был загружен.")

    filename = list(uploaded.keys())[0]
    print(f"Файл '{filename}' успешно загружен.")
    return filename

In [ ]:
# убираем results
def flatten_results(data):
    """
    Убирает ключ 'results', поднимая его содержимое (например, 'annotations')
    на один уровень выше. Все остальные поля (например, 'review_id') сохраняются.
    """
    if isinstance(data, list):
        return [flatten_results(item) for item in data]
    elif isinstance(data, dict):
        if "results" in data and isinstance(data["results"], dict):
            # Создаём новый словарь без 'results', но со всем остальным
            new_dict = {k: v for k, v in data.items() if k != "results"}
            # Добавляем содержимое 'results'
            new_dict.update(data["results"])
            return new_dict
        else:
            # Рекурсивно обрабатываем вложенные структуры
            return {k: flatten_results(v) for k, v in data.items()}
    else:
        return data

In [ ]:
def fix_categories(data):
    """
    Проходит по всем аннотациям и заменяет недопустимые категории на 'Прочие услуги'.

    Args:
        data: список словарей вида {"review_id": ..., "annotations": [...]}

    Returns:
        тот же список, но с исправленными категориями
    """
    for item in data:
        for ann in item.get("annotations", []):
            if ann.get("category") not in DEFAULT_CATEGORIES:
                print(f"⚠️ Неизвестная категория '{ann['category']}' → заменён на 'Прочие услуги'")
                ann["category"] = "Прочие услуги"
    return data

In [ ]:
import random
from collections import defaultdict

def sample_examples_by_category(data, n_per_category=3, seed=42):
    """
    Для каждой категории из DEFAULT_CATEGORIES собирает до n_per_category
    случайных примеров вида (review_id, summary).

    Args:
        data: список словарей с аннотациями
        n_per_category: сколько примеров брать на категорию
        seed: для воспроизводимости случайной выборки

    Returns:
        dict: {category: [(review_id, summary), ...]}
    """
    random.seed(seed)
    category_examples = defaultdict(list)

    for item in data:
        review_id = item["review_id"]
        for ann in item.get("annotations", []):
            cat = ann["category"]
            summary = ann.get("summary", "")
            if cat in DEFAULT_CATEGORIES:
                category_examples[cat].append((review_id, summary))

    # Оставляем только n_per_category случайных примеров на категорию
    result = {}
    for cat in DEFAULT_CATEGORIES:
        examples = category_examples[cat]
        if examples:
            sampled = random.sample(examples, min(n_per_category, len(examples)))
            result[cat] = sampled
        else:
            result[cat] = []

    return result

In [ ]:
def fix_sentiments(data):
    """
    Нормализует поле 'sentiment' в аннотациях:
    - приводит к нижнему регистру,
    - использует словарь для исправления вариантов,
    - если не найдено — ставит 'нейтральный'.

    Args:
        data: список словарей с аннотациями

    Returns:
        тот же список с исправленными сентиментами
    """
    for item in data:
        for ann in item.get("annotations", []):
            raw = ann.get("sentiment", "").strip()
            if not raw:
                ann["sentiment"] = "нейтральный"
                continue

            # Приводим к нижнему регистру
            raw_lower = raw.lower()

            # Если уже канонический — оставляем
            if raw_lower in SENTIMENTS:
                ann["sentiment"] = raw_lower
                continue

            # Ищем в маппинге
            if raw_lower in SENTIMENT_NORMALIZATION:
                ann["sentiment"] = SENTIMENT_NORMALIZATION[raw_lower]
            else:
                # Неизвестный сентимент → нейтральный
                print(f"⚠️ Неизвестный сентимент '{raw}' → заменён на 'нейтральный'")
                ann["sentiment"] = "нейтральный"
    return data

In [ ]:
def print_statistics(data):
    cat_counter = Counter()
    sent_counter = Counter()
    for item in data:
        for ann in item.get("annotations", []):
            cat_counter[ann["category"]] += 1
            sent_counter[ann["sentiment"]] += 1

    print("Категории:")
    for cat, cnt in sorted(cat_counter.items()):
        print(f"  {cat}: {cnt}")

    print("\nСентименты:")
    for sent in ["позитив", "негатив", "нейтральный"]:
        print(f"  {sent}: {sent_counter[sent]}")

In [ ]:
import pandas as pd
import json

#Загружаем csv
filename = upload_file()

# Загружаем результаты
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

Загрузите json с результатами...


Saving llm_annotations_full.json to llm_annotations_full.json
Файл 'llm_annotations_full.json' успешно загружен.


In [ ]:
# === КОНСТАНТЫ ===
DEFAULT_CATEGORIES = [
    "Карты",
    "Банкоматы",
    "Кэшбэк / Бонусы",
    "Обслуживание в офисе",
    "Вклады",
    "Кредиты",
    "Курьерская служба",
    "Приложение / сайт",
    "Служба поддержки",
    "Счета",
    "Прочие услуги"
]
SENTIMENTS = [
   "позитив", "негатив", "нейтральный"
]

SENTIMENT_NORMALIZATION = {
    "позитив": ["positive", "позитивно", "положительно", "положительный"],
    "негатив": ["negative", "негативно", "отрицательно"],
    "нейтральный": ["neutral", "нейтрально"]
}

In [ ]:
import json
from collections import Counter, defaultdict

data = flatten_results(data)
data = fix_categories(data)
data = fix_sentiments(data)
print_statistics(data)

OUTPUT_PATH = "cleaned_banki_sravni.json"

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Готово. Результат: {OUTPUT_PATH}")

⚠️ Неизвестная категория 'Кредиты / Вклады' → заменён на 'Прочие услуги'
⚠️ Неизвестный сентимент 'нейтрально' → заменён на 'нейтральный'
⚠️ Неизвестный сентимент 'нейтрально' → заменён на 'нейтральный'
Категории:
  Банкоматы: 9
  Вклады: 65
  Карты: 155
  Кредиты: 34
  Курьерская служба: 81
  Кэшбэк / Бонусы: 71
  Обслуживание в офисе: 132
  Приложение / сайт: 99
  Прочие услуги: 27
  Служба поддержки: 127
  Счета: 47

Сентименты:
  позитив: 452
  негатив: 338
  нейтральный: 57

✅ Готово. Результат: cleaned_banki_sravni.json
